In [5]:
import pandas as pd
import warnings
from pandas import Series, DataFrame
warnings.filterwarnings("ignore")

In [6]:
data_offline = pd.read_csv('ccf_offline.csv')
data_online = pd.read_csv('ccf_online.csv')

In [1]:
def get_discount_rate(s):
    if s != 'null':
        if len(s.split(':')) > 1:
            return (float(s.split(':')[0]) - float(s.split(':')[1])) / float(
                s.split(':')[0])
        elif s == 'fixed':
            return s
        else:
            return float(s)
    else:
        return 'null'


def get_coupon_requirement(s):
    if s != 'null':
        if len(s.split(':')) > 1:
            return s.split(':')[0]
        else:
            return 'null'
    else:
        return 'null'


def is_direct_use(s):
    if (len(s.split(':')) == 1) & (s != 'null'):
        return 1
    else:
        return 0


def get_week_day(s):
    if s != 'null':
        date_time = datetime.datetime.strptime(s, '%Y%m%d')
        return date_time.weekday()
    else:
        return 'null'


def get_month_day(s):
    if s != 'null':
        return int(s[-2:])
    else:
        return 'null'

In [79]:
import time, datetime


def get_coupon_feature_online(input_data):
    a = input_data[['Merchant_id', 'Coupon_id']]
    mask = input_data['Coupon_id'] != 'null'
    num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()
    num_coupon.columns = ['Merchant_id', 'Tot_coupon_sent']
    input_data1 = pd.merge(
        input_data, num_coupon, on='Merchant_id', how='left')
    mask2 = (input_data1['Date'] != 'null') & (input_data1['Coupon_id'] !=
                                               'null')
    b = input_data1[mask2].groupby(
        ['Merchant_id'])['Action'].count().reset_index()
    b.columns = ['Merchant_id', 'Num_coupon_used']
    input_data2 = pd.merge(input_data1, b, on='Merchant_id', how='left')
    l = input_data2[['Merchant_id', 'Coupon_id', 'Action']].groupby(
        ['Merchant_id', 'Coupon_id']).count().reset_index()
    mask3 = l['Coupon_id'] != 'null'
    l2 = l[mask3]
    l2['num_type'] = l2['Coupon_id'].apply(lambda x: int(x > 0))
    l3 = l2.groupby('Merchant_id')['num_type'].count()
    l4 = pd.DataFrame(l3).reset_index()
    input_data3 = pd.merge(input_data2, l4, on='Merchant_id', how='left')
    couponmask = (input_data3['Date'] != 'null') & (input_data3['Coupon_id'] !=
                                                    'null')
    coupon_used = input_data3[couponmask]
    input_data3['coupon_requirement'] = input_data3['Discount_rate'].map(
        get_coupon_requirement)
    input_data3['equivalent_discount'] = input_data3['Discount_rate'].map(
        get_discount_rate)
    input_data3['direct_use_coupon'] = input_data3['Discount_rate'].map(
        is_direct_use)
    input_data3['Date_received_week'] = input_data3['Date_received'].map(
        get_week_day)
    input_data3['Date_received_month'] = input_data3['Date_received'].map(
        get_month_day)
    output = input_data3[input_data3['Coupon_id'] != 'null']
    output.drop(['Discount_rate', 'Date_received', 'Date'], axis=1, inplace=True)
    return output

In [80]:
a=get_coupon_feature(data_online)

In [81]:
a.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,Tot_coupon_sent,Num_coupon_used,num_type,coupon_requirement,equivalent_discount,direct_use_coupon,Date_received_week,Date_received_month
0,13740231,18907,2,100017492,500:50,20160513,null,52552.0,10927.0,152.0,500,0.9,0,4,13
73,15137031,25104,2,100120354,150:50,20160613,null,31406.0,3670.0,176.0,150,0.666667,0,0,13
74,15137031,44706,2,100071973,50:5,20160317,null,1703.0,321.0,12.0,50,0.9,0,3,17
114,15137031,29007,2,100028000,30:1,20160405,null,260.0,22.0,4.0,30,0.966667,0,1,5
115,15137031,18907,2,100086665,300:50,20160406,null,52552.0,10927.0,152.0,300,0.833333,0,2,6


## For offline data

In [82]:
def get_coupon_feature_offline(input_data):
    a = input_data[['Merchant_id', 'Coupon_id']]
    mask = input_data['Coupon_id'] != 'null'
    num_coupon = a[mask].groupby(['Merchant_id']).count().reset_index()
    num_coupon.columns = ['Merchant_id', 'Tot_coupon_sent']
    input_data1 = pd.merge(input_data, num_coupon, on='Merchant_id')
    mask2 = (input_data1['Date'] != 'null') & (input_data1['Coupon_id'] !=
                                                 'null')
    b = input_data1[mask2].groupby(
        ['Merchant_id'])['Coupon_id'].count().reset_index()
    b.columns = ['Merchant_id', 'Num_coupon_used']
    input_data2 = pd.merge(input_data1, b, on='Merchant_id')
    l = input_data2[['Merchant_id', 'Coupon_id']].groupby(
        ['Merchant_id', 'Coupon_id']).count().reset_index()
    mask3 = l['Coupon_id'] != 'null'
    l2 = l[mask3]
    l2['num_type'] = l2['Coupon_id'].apply(lambda x:int(x>0))
    l3 = l2.groupby('Merchant_id')['num_type'].count()
    l4 = pd.DataFrame(l3).reset_index()
    input_data3 = pd.merge(input_data2, l4, on='Merchant_id')
    null_mask = input_data3['Distance'] != 'null'
    input_data4 = input_data3[null_mask]
    input_data4['Distance'] = input_data4['Distance'].map(lambda x: float(x))
    input_data5 = pd.DataFrame(
        input_data4.groupby('Merchant_id')['Distance'].mean()).reset_index()
    input_data6 = pd.merge(input_data3, input_data5, on='Merchant_id')
    input_data6 = input_data6.rename(columns={'Distance_y': 'Avg_Distance'})
    input_data6['equivalent_discount'] = input_data6['Discount_rate'].map(
        get_discount_rate)
    input_data6['direct_use_coupon'] = input_data6['Discount_rate'].map(
        is_direct_use)
    input_data6['coupon_requirement'] = input_data6['Discount_rate'].map(
        get_coupon_requirement)
    input_data6['Date_received_week'] = input_data6['Date_received'].map(
        get_week_day)
    input_data6['Date_received_month'] = input_data6['Date_received'].map(
        get_month_day)
    output=input_data6[input_data6['Coupon_id']!='null'] 
    output.drop(['Discount_rate', 'Date_received', 'Date'], axis=1, inplace=True)
    return output

In [83]:
b=get_coupon_feature_offline(data_offline)

In [84]:
b.head()

,User_id,Merchant_id,Coupon_id,Distance_x,Tot_coupon_sent,Num_coupon_used,num_type,Avg_Distance,equivalent_discount,direct_use_coupon,coupon_requirement,Date_received_week,Date_received_month
1,1439408,2632,8591,0,43,4,2,1.540000,0.95,0,20,2,17
2,1439408,2632,1078,0,43,4,2,1.540000,0.95,0,20,5,19
3,1439408,2632,8591,0,43,4,2,1.540000,0.95,0,20,0,13
5,1439408,2632,8591,0,43,4,2,1.540000,0.95,0,20,0,16
6,2082569,2632,1078,3,43,4,2,1.540000,0.95,0,20,4,25
7,2082569,2632,8591,3,43,4,2,1.540000,0.95,0,20,1,23
8,5162810,2632,8591,10,43,4,2,1.540000,0.95,0,20,3,4
10,5788670,2632,1078,null,43,4,2,1.540000,0.95,0,20,5,16
11,5788670,2632,8591,null,43,4,2,1.540000,0.95,0,20,2,16
12,6216806,2632,8591,0,43,4,2,1.540000,0.95,0,20,3,28
